In [294]:
import logging
logging.getLogger().setLevel(logging.WARNING)

from konlpy.corpus import kobill, kolaw
from konlpy.tag import Twitter
twitter = Twitter()

stopwords = "수 년 등 및 몇 중 네이버 뉴스".split()
stopwords += "공급 설치 조성 운영 실행 설립 확대 건설 제공 사업 실시 지원 검토 육성 추진 유치 강화 개선 구축 마련 확충 실시 개선 해소".split()
stopwords = set(stopwords)

pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True) if p[0] not in stopwords and p[1] in ('Noun', 'Verb', 'Adjective')]

In [295]:
%%time
texts_kobill = [pos(kobill.open(i).read()) for i in kobill.fileids()]
texts_kolaw = [pos(kolaw.open(i).read()) for i in kolaw.fileids()]

CPU times: user 1.32 s, sys: 31.2 ms, total: 1.36 s
Wall time: 1.44 s


In [296]:
%%time
import json

texts = []

with open('jisa_tagged.json') as data_file:    
    data = json.load(data_file)
    for voting_district in data:
        promises = voting_district['promises']
        for p in promises:
            texts.append((p['title'], pos(p['title'])))

texts_promises = [text[1] for text in texts]

CPU times: user 1.73 s, sys: 30.2 ms, total: 1.76 s
Wall time: 1.99 s


In [297]:
from gensim import corpora, models
# Create dictionary
corpus = texts_kobill + texts_kolaw + texts_promises
#corpus = texts_promises
logging.info('Dictionary from %d documents' % len(corpus))
dictionary_ko = corpora.Dictionary(corpus)
# Filter terms that occur in more than 2% of the documents
dictionary_ko.filter_extremes(no_below=0, no_above=0.02)
# Save for later use
dictionary_ko.save('ko.dict')

In [298]:
%%time
from gensim import similarities

num_topics = 100

# Generate Lsi model for promise text corpus
corpus = [dictionary_ko.doc2bow(text) for text in texts_promises]
# Train Lsi model
lsi = models.LsiModel(corpus, id2word=dictionary_ko, num_topics=num_topics)
# Transform corpus to LSI space and index it
index = similarities.MatrixSimilarity(lsi[corpus])

2017-12-21 18:02:45,384 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)


CPU times: user 984 ms, sys: 74.5 ms, total: 1.06 s
Wall time: 1.16 s


In [299]:
def find_similar(tagged_text, similarity_threshold=None):
    # Convert text to bag of words
    vec_bow = dictionary_ko.doc2bow(tagged_text)
    # Convert the query to LSI space
    vec_lsi = lsi[vec_bow]
    # Perform a similarity query against the corpus
    sims = enumerate(index[vec_lsi])
    if similarity_threshold:
        sims = filter(lambda item: item[1] >= similarity_threshold, sims)
    # Return the similarities sorted by descending score
    return sorted(sims, key=lambda item: -item[1])
    
# Try to match a few existing documents
for text in texts[100:111]:
    sims = find_similar(text[1])
    print("Most similar to %s (%s)" % (text[0], ' '.join(text[1])))
    for key, score in sims[1:4]:
        key = int(key)
        print('%.2f - %s' % (score, texts[key][0]))

Most similar to 동북아 대도시 대기환경협의체 구성 및 대기개선 노하우 공유 (동북아/Noun 대도시/Noun 대기/Noun 환경/Noun 협의/Noun 체/Noun 구성/Noun 대기/Noun 노하우/Noun 공유/Noun)
0.92 - 영남권대기환경청 설립 추진
0.75 - 노사민정협의체 활성화로 상생기업 환경 구축
0.70 - 문화거버넌스 구성 및 운영
Most similar to 초미세먼지 노출 예방 시스템 구축 (초미세먼지/Noun 노출/Noun 예방/Noun 시스템/Noun)
0.89 - 공유경제 지원시스템 구축
0.87 - 지역 공기질 측정 및 실시간 공개시스템 구축
0.86 - 9-1 대중교통 시스템의 획기적 개선 (촘촘한 노선 구축, 심야버스 운행 등)
Most similar to 생활권 주변 10분거리 공원 조성 (생활권/Noun 주변/Noun 분/Noun 거리/Noun 공원/Noun)
0.92 - 공원과 산책로 칼로리 게시판 후원합니다
0.92 - 시민체감형 도시공원, 녹지관리
0.92 - 대통령 기념공원 조성
Most similar to 유아숲 체험장 50개소 설치 등 생애주기별 힐링공원 조성 (유아/Noun 숲/Noun 체험/Noun 개다/Verb 생애/Noun 주기/Noun 별/Noun 힐링/Noun 공원/Noun)
0.81 - 생애주기형 도시숲 조성
0.77 - 힐링 아일랜드 조성
0.73 - 도시숲 확대
Most similar to 걷고 싶은 서울길’ 완성 (걷다/Verb 싶다/Verb 서울/Noun 길/Noun 완성/Noun)
0.94 - 서울공예박물관’ 건립
0.94 - 서울시네마테크’ 건립
0.94 - 서울전역 와이파이존 구축
Most similar to 학교 80개교와 사회복지시설에 ‘싱싱텃밭’ 조성 (학교/Noun 개교/Noun 사회/Noun 복지/Noun 시설/Noun 싱싱/Noun 텃밭/Noun)
0.99 - 50+ 경험과 노하우를 배우는 온라인 인생학교(TED) 조성
0.89 - 초.중,특수학교 친환경 학교

In [282]:
%%time
# Find similarities of all documents with each other and see how many matches higher than x score there are
# Used to figure out a threshold that regards enough documents as matched
threshold = 0.65
counts = []
counts_self_no_match = []
for idx, text in enumerate(texts):
    similarities = np.array(find_similar(text[1]))
    # Remove self-similarity
    id_idx = np.where(similarities[:, 0]==idx)[0][0]
    similarities = np.delete(similarities, id_idx, axis=0)
    # Record when self wasn't the best match (it is usally the 2nd best in these cases)
    if id_idx != 0:
        counts_self_no_match.append((idx, id_idx))
    # Count number of docs with similarity >= threshold
    count = np.sum(similarities[:,1]>=threshold)  # Skip the first row since it will be the queried text itself
    counts.append(count)

counts = np.array(counts)
counts_self_no_match = np.array(counts_self_no_match)
print('Number of matches per document: max %d, min %d, mean %.2f, median %d' % (np.max(counts), np.min(counts), np.mean(counts), np.median(counts)))
print('%d documents didn\'t match with themself as their best match' % (len(counts_self_no_match)))
print('  but %d of them matched at pos 1.' % (np.sum(counts_self_no_match[:,1]==1)))

no_match_count = np.sum(counts==0)
print('%d (%d%%) with 0 matches' % (no_match_count, 100*no_match_count/len(counts)))

Number of matches per document: max 33, min 0, mean 9.83, median 9
55 documents didn't match with themself as their best match
  but 43 of them matched at pos 1.
124 (5%) with 0 matches
CPU times: user 9.14 s, sys: 208 ms, total: 9.35 s
Wall time: 10.5 s


In [293]:
# Try some queries that don't match existing documents

def match(title):
    print("Most similar to %s" % title)
    threshold = 0.65
    for posi, item in enumerate(find_similar(pos(title), threshold)):
        key = int(item[0])
        score = item[1]
        print('%2d. %.2f - %s' % (posi+1, score, texts[key][0]))

match('임대주택')
match(' 임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토')


Most similar to 임대주택
 1. 1.00 - 소정면 임대주택 건설
 2. 0.86 - 신혼부부‧학생‧일인가구 등 수요대응형 주택공급 및 임대주택 지속 공급
 3. 0.82 - 민간임대주택 지원을 위한 국민주택기금 및 이자지원 확대
 4. 0.76 - 다세대주택 범죄자! ‘특수 형광물질’이 다 잡아냅니다
 5. 0.76 - 전용면적 30~60m² 소형주택 20만호 공급 지원
 6. 0.75 - 노후 주택 녹슨 상수도관 개량 지원
 7. 0.72 - 공동체 주택건설
Most similar to  임대주택 등록하면 세금·건보료 감면…2020년 등록 의무화 검토
 1. 0.82 - 소정면 임대주택 건설
 2. 0.75 - 신혼부부‧학생‧일인가구 등 수요대응형 주택공급 및 임대주택 지속 공급
 3. 0.72 - 민간임대주택 지원을 위한 국민주택기금 및 이자지원 확대
 4. 0.70 - 노후 주택 녹슨 상수도관 개량 지원
 5. 0.70 - 전용면적 30~60m² 소형주택 20만호 공급 지원
 6. 0.69 - 공동체 주택건설
 7. 0.68 - 주택임대차 표준계약서 의무사용 법제화 건의
 8. 0.68 - 다세대주택 범죄자! ‘특수 형광물질’이 다 잡아냅니다
